In [1]:
import os as os
import sys as sys
import re as re
import pandas as pd
import numpy as np
import json as json
import pickle
import urllib as urllib
import zlib as zlib
import base64 as base64
from requests import Request, Session
import requests
from numpy import trapz
import itertools
import zlib as zlib
import base64 as base64
import pickle
import pathlib
from tqdm import tqdm
from datetime import datetime
import time
import copy
from Bio import SeqIO
import inspect

pd.options.display.max_columns = 100
pd.options.display.min_rows = None
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 10000

DATA_PATH = pathlib.Path('/data1/home/adpatter/gene-to-phenotype-predictions/adpatter/data/')
PROTEIN_PATH = DATA_PATH.joinpath('uniprot-compressed_true_download_true_format_json_query__28_28organi-2023.02.20-21.42.00.91.json')
RESULT_PATH = DATA_PATH.joinpath('motifs.pkl')

In [2]:
with open(PROTEIN_PATH, 'r') as f:

    data = json.load(f)

results = data['results']

In [3]:
records = []

for result in results:
    feature_seqs = []
    start = 0
    end = 0
    sequence = None
    features = None
    if 'sequence' in result:
        sequence = result['sequence']['value']
        if 'features' in result:
            features = result['features']
            for feature in features:
                if feature['type'] == 'Motif':
                    if 'location' in feature:
                        location = feature['location']
                        start = location['start']['value']
                        end = location['end']['value']
                        if start and end:
                            feature_seq = sequence[start-1:end]
                            feature_seqs.append(feature_seq)

            if len(feature_seqs) > 0:
                records.append({
                    'UniqueIdentifier': result['primaryAccession'],
                    'EntryName': result['uniProtkbId'],
                    'motifs':' '.join(feature_seqs)
                })

df = pd.DataFrame(records)

df.to_pickle(RESULT_PATH)

In [2]:
df = pd.read_pickle(RESULT_PATH)

df

,UniqueIdentifier,EntryName,motifs
0,A0A0B4J1F4,ARRD4_MOUSE,PPNY PPLY
1,A0A0N4SVM5,A0A0N4SVM5_MOUSE,PTFDTMGLREDLLRGIYAYGFEKPSAIQQ
2,A0A0N4SVP8,A0A0N4SVP8_MOUSE,LTFDTMGLREDLLRGIYAYGFEKPSAIQQ
3,A0A1D9BZF0,GCNA_MOUSE,SGSS CVVI VCEI LIVI
4,A0A1W2P872,NOVA2_MOUSE,KRPLETPPEVVCTKRSN
5,A0A286YE25,A0A286YE25_MOUSE,PAGRRKKK VGFALLRLL RKKK
6,A0A3B2WC92,A0A3B2WC92_MOUSE,PAGRRKKK VGFALLRLL RKKK
7,A2A432,CUL4B_MOUSE,KKRK
8,A2A6C4,SPP2C_MOUSE,PAL
9,A2A6M5,CACO2_MOUSE,IMVV ASWE
